In [2]:
import openai
import json
import numpy as np
from numpy.linalg import norm

api_key = open("api_key").read()
openai.api_key = api_key

In [2]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

def cos_sim(a, b):
   return np.dot(a, b)/(norm(a)*norm(b))

def save_json(data, filepath=r'new_data.json'):
   with open(filepath, 'w') as fp:
      json.dump(data, fp, indent=4)

In [ ]:
def merge_paragraphs(sentences):
    sentence_list = [" ".join(sentence_word_list) for sentence_word_list in sentences] # merge the words into sentences
    paragraph = " ".join(sentence_list)
    return paragraph

In [3]:
# All_articles = json.load(open("data/raw/AllTheNews/events/2016_10p.json"))
All_articles = json.load(open("data/result/AllTheNews/network/articles.json"))
existing_embeddings = json.load(open("data/raw/AllTheNews/embeddings/2016_10p.json"))

In [4]:
all_article_ids = list(All_articles.keys())
existing_article_ids = [article['id'] for article in existing_embeddings]
target_article_ids = list(set(all_article_ids) - set(existing_article_ids))
print(len(existing_embeddings), len(all_article_ids), len(target_article_ids))

7637 7604 1


In [5]:
# All The News
for article_id in target_article_ids:
    article = All_articles[article_id]
    article['embedding'] = get_embedding(article['summary'])
    existing_embeddings.append({
        "id": article_id,
        "summary": article['summary'],
        "content": article['content'],
        "embedding": article['embedding'],
    })
    print("add: ", len(existing_embeddings))
print(len(existing_embeddings))
# save_json(existing_embeddings, "data/raw/AllTheNews/embeddings/2016_10p.json")

add:  7638
7638


In [12]:
save_json(existing_embeddings, "data/raw/AllTheNews/embeddings/2016_10p.json")

In [4]:
All_articles = json.load(open("data/raw/AllTheNews/events/2016_10p.json"))
All_article_embeddings = json.load(open("data/raw/AllTheNews/embeddings/2016_10p.json"))
print(len(All_article_embeddings), len(All_articles))

7548 7548


In [ ]:
# RAMS
# embeddings_dict = {candidate: get_embedding(candidate_paragraphs[candidate]) for candidate in candidates}
for event_id, event in event_dict.items():
    print(event_id, len(event_dict))
    event['embedding'] = get_embedding(merge_paragraphs(event['content']))
save_json(event_dict, "data/result/RAMS/gpt_biHgraph_dev/hyperedges_w_embeddings.json")

In [ ]:
def reorder_hyperedge_id(hyperedges_w_embeddings_dict):
    ordered_dict = {}
    for key, value in hyperedges_w_embeddings_dict.items():
        id_split = key.split('-')
        doc_id = id_split[0]
        triggers = id_split[1:]
        triggers.sort()
        ordered_dict[doc_id + '-' + '-'.join(triggers)] = value
    return ordered_dict
hyperedges_w_embeddings_dict = json.load(open("data/result/RAMS/gpt_biHgraph_dev/hyperedges_w_embeddings.json"))
ordered_dict = reorder_hyperedge_id(hyperedges_w_embeddings_dict)
save_json(ordered_dict, 'data/result/RAMS/gpt_biHgraph_dev/hyperedges_w_embeddings_ordered.json')

In [3]:
server_embeddings = json.load(open("data/result/AllTheNews/network/server/embeddings.json"))
embeddings_dict = { doc['doc_id']: doc for doc in server_embeddings}
frontend_data = json.load(open('data/result/AllTheNews/network/server/frontend_2.json'))
article_node_ids = [node['id'] for node in frontend_data['nodes'] if node['type'] == 'article']
for article_id in article_node_ids:
    if article_id not in embeddings_dict:
        print(article_id)
        # embeddings_dict[article_id] = {
        #     'doc_id': article_id,
        #     'embedding': get_embedding(All_articles[article_id]['summary']),
        # }

